# 8.1 Fehlende Daten

Realistische Datensätze sind oft unvollständig. In einer Umfrage hat eine Person
mit einer Frage nichts anfangen können und daher nichts angekreuzt. Ein
Messsensor an der Produktionsanlage ist abends ausgefallen, was erst am nächsten
Morgen bemerkt wurde. Die Mitarbeitenden einer Arztpraxis sind im Urlaub und
lassen die Meldung der verabreichten Impfungen noch bis nach dem Urlaub liegen.
Es gibt viele Gründe, warum Datensätze unvollständig sind. In diesem Abschnitt
beschäftigen eir uns damit, fehlende Daten aufzuspüren und lernen einfache
Methoden kennen, damit umzugehen.

## Lernziele

```{admonition} Lernziele
:class: goals
* Sie können in einem Datensatz mit **isnull()** fehlende Daten aufspüren und
  analysieren.
* Sie kennen die beiden grundlegenen Strategien, mit fehlenden Daten umzugehen:
  * **Elimination** (Löschen) und
  * **Imputation** (Vervollständigen).
* Sie können Daten gezielt mit **drop()** löschen.
* Sie können fehlende Daten mit **fillna()** ersetzen.
```

## Fehlende Daten aufspüren mit isnull()

Wir arbeiten im Folgenden mit einem echten Datensatz der Verkaufsplattform
[Autoscout24.de](https://www.autoscout24.de), der Verkaufsdaten zu 1000 Autos
enthält. Sie können die csv-Datei hier herunterladen {download}`Download
autoscout24_fehlende_daten.csv
<https://gramschs.github.io/book_ml4ing/data/autoscout24_fehlende_daten.csv>`
und in das Jupyter Notebook importieren. Alternativ können Sie die csv-Datei
auch über die URL importieren, wie es in der folgenden Code-Zelle gemacht wird.

In [1]:
import pandas as pd

url = 'https://gramschs.github.io/book_ml4ing/data/autoscout24_fehlende_daten.csv'
daten = pd.read_csv(url)

daten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marke                 1000 non-null   object 
 1   Modell                1000 non-null   object 
 2   Farbe                 964 non-null    object 
 3   Erstzulassung         1000 non-null   object 
 4   Jahr                  1000 non-null   object 
 5   Preis (Euro)          986 non-null    object 
 6   Leistung (kW)         1000 non-null   object 
 7   Leistung (PS)         1000 non-null   object 
 8   Getriebe              1000 non-null   object 
 9   Kraftstoff            1000 non-null   object 
 10  Verbrauch (l/100 km)  892 non-null    object 
 11  Verbrauch (g/km)      1000 non-null   object 
 12  Kilometerstand (km)   999 non-null    float64
 13  Bemerkungen           1000 non-null   object 
dtypes: float64(1), object(13)
memory usage: 109.5+ KB


Wir hatten bereits festgestellt, dass die Anzahl der `non-null`-Einträge für die
verschiedenen Merkmale unterschiedlich ist. Offensichtlich ist nur bei 964 Autos eine Farbe eingetragen und 892 Autos
die Eigenschaft »Verbrauch (l/100 km)« gültig und auch der »Kilometerstand (km)«
enthält nur 999 gültige Einträge. Welche das sind, können wir mit der Methode
`isnull()` bestimmen. Die Methode liefert ein Pandas DataFrame zurück, das
True/False-Werte enthält. True steht dabei dafür, dass ein Wert fehlt bzw. mit
dem Eintrag 'NaN' gekennzeichnet ist (= not a number). Weitere Details finden
Sie in der [Pandas-Dokumentation →
isnull()](https://pandas.pydata.org/docs/reference/api/pandas.isnull.html).

In [2]:
daten.isnull()

,Marke,Modell,Farbe,Erstzulassung,Jahr,Preis (Euro),Leistung (kW),Leistung (PS),Getriebe,Kraftstoff,Verbrauch (l/100 km),Verbrauch (g/km),Kilometerstand (km),Bemerkungen
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,False,False,False,False,False,False,False,False,False,False,False,False,False,False
996,False,False,False,False,False,False,False,False,False,False,False,False,False,False
997,False,False,False,False,False,False,False,False,False,False,False,False,False,False
998,False,False,False,False,False,False,False,False,False,False,False,False,False,False


Bereits in der zweiten Zeile befindet sich ein Auto, bei dem das Merkmal
»Verbrauch (l/100 km)« nicht gültig ist (ggf. müssen Sie weiter nach rechts
scrollen), den dort steht `True`. Wir betrachten uns diesen Eintrag:

In [3]:
daten.loc[1,:]

Marke                                                            bmw
Modell                                                        BMW X1
Farbe                                                           blau
Erstzulassung                                                06/2021
Jahr                                                            2021
Preis (Euro)                                                   42830
Leistung (kW)                                                    162
Leistung (PS)                                                    220
Getriebe                                                   Automatik
Kraftstoff                                                    Hybrid
Verbrauch (l/100 km)                                             NaN
Verbrauch (g/km)                                    49 km Reichweite
Kilometerstand (km)                                          29700.0
Bemerkungen             xDrive25e M Sport HUD Navi Plus DA+ Pano AHK
Name: 1, dtype: object

Bei dem Auto handelt es sich um einen Hybrid, vielleicht wurde deshalb der
»Verbrauch (l/100 km)« nicht angegeben. Ist das vielleicht auch bei den anderen
Autos der Grund? Wir speichern zunächst die isnull()-Datenstruktur in einer
eigenen Variable ab und ermitteln zunächst, wie viele Autos keinen gültigen
Eintrag bei diesem Merkmal haben. Dazu nutzen wir aus, dass der boolesche Wert
`False` bei Rechnungen als 0 interpretiert wird und der boolesche Wert `True`
als 1. Die Methode `.sum()` summiert pro Spalte alle Werte, so dass sie direkt
die Anzahl der ungültigen Werte pro Spalte liefert.

In [4]:
fehlende_daten = daten.isnull()

fehlende_daten.sum()

Marke                     0
Modell                    0
Farbe                    36
Erstzulassung             0
Jahr                      0
Preis (Euro)             14
Leistung (kW)             0
Leistung (PS)             0
Getriebe                  0
Kraftstoff                0
Verbrauch (l/100 km)    108
Verbrauch (g/km)          0
Kilometerstand (km)       1
Bemerkungen               0
dtype: int64

Jetzt lassen wir uns diese 108 Autos anzeigen, bei denen ungültige Werte beim
»Verbrauch (l/100 km)« angegeben wurden. Dazu nutzen wir die True-Werte in der
Spalte `Verbrauch (l/100 km)` als Filter für den ursprünglichen Datensatz.
Zumindest die ersten 20 Autos lassen wir uns dann mit der `.head(20)`-Methode
anzeigen.

In [5]:
autos_mit_fehlendem_verbrauch_pro_100km = daten[ fehlende_daten['Verbrauch (l/100 km)'] == True ]
autos_mit_fehlendem_verbrauch_pro_100km.head(20)

,Marke,Modell,Farbe,Erstzulassung,Jahr,Preis (Euro),Leistung (kW),Leistung (PS),Getriebe,Kraftstoff,Verbrauch (l/100 km),Verbrauch (g/km),Kilometerstand (km),Bemerkungen
1,bmw,BMW X1,blau,06/2021,2021,42830,162,220,Automatik,Hybrid,NaN,49 km Reichweite,29700.0,xDrive25e M Sport HUD Navi Plus DA+ Pano AHK
5,peugeot,Peugeot 308,blau,08/2019,2019,16990,96,131,Schaltgetriebe,Benzin,NaN,- (g/km),72500.0,SW PureTech 130*ALLURE*CARPLAY*3D-NAVI*ALU*
31,mercedes-benz,Mercedes-Benz B 250,weiß,10/2021,2021,NaN,160,218,Automatik,Benzin,NaN,0 g/km,22000.0,e*HYBRID*NAVI*LED*TEMPOMAT*AMG-PAKET
68,chrysler,Chrysler Pacifica,schwarz,06/2022,2022,54490,214,291,Automatik,Benzin,NaN,- (g/km),28512.0,"Touring L, 3.6l V6, NEUES MODELL,CARFAX"
77,seat,SEAT Alhambra,blau,04/2019,2019,29990,130,177,Automatik,Diesel,NaN,179 g/km,85200.0,2.0 TDI DSG Style*Navi*Bi-Xenon*Panoram Style
79,bmw,BMW 520,silber,07/2019,2019,37890,140,190,Automatik,Diesel,NaN,0 g/km,67467.0,d xDrive Touring Luxury Line NaviProf.HUD.AHK
81,volvo,Volvo S90,NaN,08/2020,2020,41900,246,334,Automatik,Benzin,NaN,- (g/km),46800.0,T6 AWD R-Design ACC AutoPilot LED Panorama 3
87,mercedes-benz,Mercedes-Benz,rot,06/2022,2022,42570,140,190,Automatik,Diesel,NaN,-/-,7900.0,Sprinter 319 CDI Pritsche DoKa Leder S&S 7-Sitze
88,nissan,Nissan Qashqai,schwarz,10/2020,2020,25600,117,159,Automatik,Benzin,NaN,0 g/km,45915.0,Tekna PLUS ACC+LED+PANO+360°+Park Assist
91,volkswagen,Volkswagen Golf,schwarz,03/2015,2015,NaN,169,230,Schaltgetriebe,Benzin,NaN,- (g/km),99000.0,GTI Performance 2.0 TSI 6-Gang GTI Performance...


Bemerkung: Der Vergleich `== True` ist redundant und kann auch weggelassen werden.

Beim Kraftstoff werden alle möglichen Angaben gemacht: Hybrid, Benzin, Diesel
und Elektro. Wir müssten jetzt systematisch den fehlenden Angaben nachgehen. Für
Elektrofahrzeuge und ggf. Hybridautos ist die Angabe »Verbrauch (l/ 100 km)«
unsinnig. Aber das zweite Auto mit der Nr. 5 wird mit Benzin betrieben, da
scheint Nachlässigkeit beim Ausfüllen der Merkmale vorzuliegen. Beim fünften
Auto mit der Nr. 77 ist zwar der »Verbrauch (l/100 km)« nicht angegeben, aber
dafür der »Verbrauch (g/km)«. Daraus könnten wir den »Verbrauch (l/100 km)«
abschätzen und den fehlenden Wert ergänzen. Es gibt verschiedene Strategien, mit
fehlenden Daten umzugehen. Die beiden wichtigsten Verfahren zum Umgang mit
fehlenden Daten sind

1. Löschen (Elimination) und
2. Vervollständigung (Imputation).

Bei Elimination werden Datenpunkte (Autos) und/oder Merkmale gelöscht. Bei
Imputation (Vervollständigung) werden die fehlenden Werte ergänzt. Beide
Verfahren werden wir nun etwas detaillierter betrachten.

## Löschen (Elimination) mit drop()

Bei der Elimination (Löschen) können wir filigran vorgehen oder die
Holzhammer-Methode verwenden. Beispielsweise könnten wir entscheiden, das
Merkmal »Verbrauch (l/100 km)« komplett zu löschen und einfach nur den
»Verbrauch (g/km)« zu berücksichtigen. Aber ein kurzer Blick auf die Daten hatte
ja bereits gezeigt, dass diese Werte auch nur unzuverlässig gefüllt waren, auch
wenn sie technisch gültig sind. Wir löschen beide Merkmale. Dazu benutzen wir
die Methode `drop()` mit dem zusätzlichen Argument `columns=['Verbrauch (l/
100 km)', 'Verbrauch (g/km)']`. Da wir gleich zwei Spalten aufeinmal eliminieren
möchten, müssen wir die Spalten (Columns) als Liste übergeben. Danach überprüfen
wir mit der Methode `.info()`, ob das Löschen geklappt hat.

In [6]:
daten.drop(columns=['Verbrauch (l/100 km)', 'Verbrauch (g/km)'])
daten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marke                 1000 non-null   object 
 1   Modell                1000 non-null   object 
 2   Farbe                 964 non-null    object 
 3   Erstzulassung         1000 non-null   object 
 4   Jahr                  1000 non-null   object 
 5   Preis (Euro)          986 non-null    object 
 6   Leistung (kW)         1000 non-null   object 
 7   Leistung (PS)         1000 non-null   object 
 8   Getriebe              1000 non-null   object 
 9   Kraftstoff            1000 non-null   object 
 10  Verbrauch (l/100 km)  892 non-null    object 
 11  Verbrauch (g/km)      1000 non-null   object 
 12  Kilometerstand (km)   999 non-null    float64
 13  Bemerkungen           1000 non-null   object 
dtypes: float64(1), object(13)
memory usage: 109.5+ KB


Leider hat der Befehl `drop()` nicht funktioniert! Was ist da los? Python und
Pandas verfolgen das Programmierparadigma »Explizit ist besser als implizit!«
Daher werden zwar werden durch den `drop()`-Befehl die beiden Spalten gelöscht,
aber der Datensatz `daten` selbst bleibt aus Sicherheitsgründen unverändert.
Möchten wir den Datensatz mit den gelöschten Merkmalen weiter verwenden, müssen
wir ihn in einer neuen Variable speichern oder die alte Variable `daten` damit
überschreiben. Wir nehmen eine neue Variable namens `daten_ohne_verbrauch`.

In [7]:
daten_ohne_verbrauch = daten.drop(columns=['Verbrauch (l/100 km)', 'Verbrauch (g/km)'])
daten_ohne_verbrauch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Marke                1000 non-null   object 
 1   Modell               1000 non-null   object 
 2   Farbe                964 non-null    object 
 3   Erstzulassung        1000 non-null   object 
 4   Jahr                 1000 non-null   object 
 5   Preis (Euro)         986 non-null    object 
 6   Leistung (kW)        1000 non-null   object 
 7   Leistung (PS)        1000 non-null   object 
 8   Getriebe             1000 non-null   object 
 9   Kraftstoff           1000 non-null   object 
 10  Kilometerstand (km)  999 non-null    float64
 11  Bemerkungen          1000 non-null   object 
dtypes: float64(1), object(11)
memory usage: 93.9+ KB


Ein weiterer Datenpunkt weist einen ungültigen Eintrag für den »Kilometerstand
(km)« auf. Schauen wir zunächst nach, um welches Auto es sich handelt.

In [8]:
daten_ohne_verbrauch[ daten_ohne_verbrauch['Kilometerstand (km)'].isnull() ]

,Marke,Modell,Farbe,Erstzulassung,Jahr,Preis (Euro),Leistung (kW),Leistung (PS),Getriebe,Kraftstoff,Kilometerstand (km),Bemerkungen
708,peugeot,Peugeot 508,schwarz,37.500 km,12/2020,ACC,124 g/km,"4,4 l/100 km",Automatik,Automatik,NaN,GT BlueHDi 180 EAT8 Panoramadach


Bei den Einträgen des Autos sind noch mehr Probleme ersichtlich. Die
Erstzulassung war sicherlich nicht bei 37.500 km und das Jahr ist nicht 12/2020.
Wir können jetzt diesen Datenpunkt löschen oder den Datenpunkt reparieren.
Zunächst einmal der Code zum Löschen des Datenpunktes. Standardmäßig löscht die
`drop()`-Methode ohnehin Zeilen, also Datenpunkte, so dass wir ohne weitere
Optionen den Index der zu löschenden Datenpunkte angeben. Diesmal verwenden wir
die alte Variable um den reduzierten Datensatz zu speichern.

In [9]:
daten_ohne_verbrauch = daten_ohne_verbrauch.drop(708)
daten_ohne_verbrauch.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999 entries, 0 to 999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Marke                999 non-null    object 
 1   Modell               999 non-null    object 
 2   Farbe                963 non-null    object 
 3   Erstzulassung        999 non-null    object 
 4   Jahr                 999 non-null    object 
 5   Preis (Euro)         985 non-null    object 
 6   Leistung (kW)        999 non-null    object 
 7   Leistung (PS)        999 non-null    object 
 8   Getriebe             999 non-null    object 
 9   Kraftstoff           999 non-null    object 
 10  Kilometerstand (km)  999 non-null    float64
 11  Bemerkungen          999 non-null    object 
dtypes: float64(1), object(11)
memory usage: 101.5+ KB


Wie Sie in der [Dokumentation Scikit-Learn →
drop()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)
nachlesen können, gibt es zum expliziten Überschreiben der alten Variable auch
die Alternative, die Option `inplace=True` zu setzen. Welche Option Sie nutzen,
ist Geschmackssache.

Jetzt sind alle Einträge des Datensatzes gültig, gültig im technischen Sinne.

In [10]:
daten_ohne_verbrauch.isnull().sum()

Marke                   0
Modell                  0
Farbe                  36
Erstzulassung           0
Jahr                    0
Preis (Euro)           14
Leistung (kW)           0
Leistung (PS)           0
Getriebe                0
Kraftstoff              0
Kilometerstand (km)     0
Bemerkungen             0
dtype: int64

Ob alle Angaben plausibel sind, ist nicht gesagt. Bei dem Peugeot mit dem Index
708 hatten wir ja gesehen, dass bei der Erstzulassung eine Kilometerangabe
stand. Tatsächlich gab es bereits erste Hinweise darauf, dass manche Werte
technisch gültig, aber nicht plausibel sind. Die Spalte mit dem Jahr
beispielsweise wurde beim Import als Datentyp Object klassifiziert. Zu erwarten
wäre jedoch der Datentyp Integer gewesen. Schauen wir noch einmal in den
ursprünglichen Datensatz hinein.

In [11]:
daten['Jahr'].unique()

array(['2008', '2021', '2018', '2007', '2013', '2019', '2014', '2020',
       '2015', '2000', '2016', '2022', '2023', '2017', '2006', '2010',
       '2012', '1999', '2005', '2004', '2009', '2002', '2011', '2003',
       '1996', '2001', '1997', '12/2020'], dtype=object)

Da bei dem Peugeot mit dem Index 708 das Jahr fälschlicherweise mit `12/2020`
angegeben wurde, hat dieser eine Text-Eintrag dazu geführt, dass die komplette
Spalte als Object klassifiziert wurde und nicht als Integer. Daher müssen stets
weitere Plausibilitätsprüfungen durchgeführt werden, bevor die Daten genutzt
werden, um statistische Aussagen zu treffen oder ein ML-Modell zu trainieren.

## Vervollständigung (Imputation) mit fillna()

Auch bei den Angaben zur Schaltung fehlen Einträge. Zum Beispiel die Zeile mit
dem Index 243 ist unvollständig.

In [12]:
#print(data_raw.loc[243, :])

Diesmal entscheiden wir uns dazu, diese Eigenschaft nicht wegzulassen.
ML-Verfahren brauchen aber immer einen gültigen Wert und nicht NaN. Wir ersetzen
die fehlenden Werte durch den Eintrag 'not defined'. Genausogut könnten wir auch
'keine Angabe' oder 'nada' oder was auch immer nehmen. Dazu benutzen wir die
Methode `fillna()` (siehe [Pandas-Dokumentation →
fillna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)).

In [13]:
#data_raw.loc[:, 'gear'] = data_raw.loc[:, 'gear'].fillna(value='not defined')

Wenn wir uns jetzt noch einmal Zeile 243 ansehen, sehen wir, dass `fillna()`
funktioniert hat.

In [14]:
#print(data_raw.loc[243,:])

Bei den PS-Zahlen haben wir ebenfalls nicht vollständige Daten vorliegen.
Diesmal haben wir nicht diskrete Werte wie 'Schaltwagen' oder 'Automatik',
sondern numerische Werte. Daher bietet es sich hier eine zweite Methode der
Ersetzung an. Wenn wir überall da, wo wir keine PS-Zahlen vorliegen haben, den
Mittelwert der vorhandenen PS-Zahlen einsetzen, machen wir zumindest den
Mittelwert des gesamten Datensatzes nicht kaputt. Besser wäre natürlich zu
versuchen, die fehlenden Daten zu recherchieren. Oder aber mittels linearer
Regression die fehlenden Werte zu schätzen und dann zu ergänzen. Als erste
Näherung nehmen wir jetzt den Mittelwert der vorhandenen Daten.

In [15]:
#mittelwert = data_raw.loc[: , 'hp'].mean()
#print('Der Mittelwert der vorhandenen PS-Zahlen ist: {:.2f}'.format(mittelwert))

#data_raw.loc[:, 'hp'] = data_raw.loc[:, 'hp'].fillna(mittelwert)

## Zusammenfassung

Ein wichtiger Teil eines ML-Projektes beschäftigt sich mit der Aufbereitung der
Daten für die ML-Algorithmen. Dabei ist es nicht nur wichtig, in großen
Datensammlungen fehlende Einträge aufspüren zu können, sondern ein Gespür dafür
zu entwickeln, wie mit den fehlenden Daten angesetzt werden sollen. Die
Strategien hängen dabei von der Anzahl der fehlenden Daten und ihrer Bedeutung
ab. Häufig werden unvollständige Daten aus der Datensammlung gelöscht oder
numerische Einträge durch den Mittelwert der vorhandenen Daten ersetzt.